In [2]:

import pymongo
import pandas as pd
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Dependencies
import requests
db=client.veilofignorancedb
db.veilofignorance.drop()
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [4]:
stateID = {'Arkansas':['05','AR'], 'California':['06','CA'], 'Illinois':['17','IL'], 'Kansas':['20','KS'], 'Mississippi':['28','MS'], 'Ohio':['39','OH'], 
           'Texas':['48','TX'], 'Alabama':['01','AL'], 'Iowa':['19','IA'], 'Louisiana':['22','LA'],'Minnesota':['27','MN'], 'Missouri':['9','MO'], 
           'Nebraska':['31','NE'], 'Arizona':['04','AZ'], 'Colorado':['08','CO'], 'Indiana':['18','IN'], 'Michigan':['26','MI'],
           'Montana':['30','MT'], 'New York':['36','NY'], 'Oregon':['41','OR'], 'Virginia':['51','VA'], 'Wyoming':['56','WY'],'North Carolina':['37','NC'], 
           'Oklahoma':['40','OK'], 'Tennessee':['47','TN'], 'Wisconsin':['55','WI'], 'Alaska':['02','AK'], 'Vermont':['50','VT'], 'North Dakota':['38','ND'],
           'Georgia':['13','GA'], 'Maine':['23','ME'], 'Rhode Island':['44','RI'], 'West Virginia':['54','WV'], 'Idaho':['16','ID'],'South Dakota':['46','SD'],
           'New Mexico':['35','NM'], 'Washington':['53','WA'], 'Pennsylvania':['42','PA'], 'Florida':['12','FL'], 'Utah':['49','UT'], 'Kentucky':['21','KY'],
           'New Hampshire':['33','NH'], 'South Carolina':['45','SC'], 'Nevada':['32','NV'], 'Hawaii':['15','HI'], 'New Jersey':['34','NJ'],'Connecticut':['09','CT']
           , 'Maryland':['24','MD'], 'Massachusetts':['25','MA'], 'Delaware':['10','DE'], 'District of Columbia':['11','DC'], 'United States':['100','US']}

In [5]:

def statescrape(state, stateID):

    source = 'https://www.census.gov/quickfacts/fact/table/' + str(state)+ '/PST045218'
    html = urlopen(source)
    soup = BeautifulSoup(html, 'html.parser')

    fulllist = []

    for i in soup.find_all('td'):
    
        fulllist.append(i.text)
     
    headinglist = []
    datalist = []

    for i in range(len(fulllist)):
        if i%2 == 0:
        
            x = fulllist[i].strip('\n')
            formatted = x.strip('\ue840\ue83f\n')
            headinglist.append(formatted)
        
        else:
        
            x = fulllist[i].strip('\n')
            formatted = x.strip('\ue840\ue83f\n') 
            datalist.append(formatted)    

    truedict = dict(zip(headinglist, datalist))  
    truedict['State'] = state
    truedict['StateID'] = stateID
    statedf = pd.DataFrame(truedict, index = [state])
    onestatedf = statedf
    
    return onestatedf

In [6]:

dflist = pd.DataFrame()
count = 0
for key, value in stateID.items():
    
    #print(count, key, value)
    
    scraped_df = statescrape(value[1], value[0])
    
    dflist = dflist.append(scraped_df)
    count +=1

In [25]:
records = dflist.to_dict(orient = 'records')
records

dflist.head()

,"Population estimates, July 1, 2018, (V2018)","Population estimates, July 1, 2017, (V2017)","Population estimates base, April 1, 2010, (V2018)","Population estimates base, April 1, 2010, (V2017)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2018, (V2018)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2017, (V2017)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent",...,"Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010",FIPS Code,State,StateID
AR,"3,013,825","3,004,279","2,916,028","2,916,031",3.4%,3.0%,"2,915,918",6.4%,23.5%,16.6%,...,"75,962","35,982","189,029","25,915","192,988",56.0,"52,035.48",05,AR,05
CA,"39,557,045","39,536,653","37,254,523","37,254,518",6.2%,6.1%,"37,253,956",6.3%,22.9%,13.9%,...,"1,320,085","1,619,857","1,819,107","252,377","3,176,341",239.1,"155,779.22",06,CA,06
IL,"12,741,080","12,802,023","12,831,572","12,831,565",-0.7%,-0.2%,"12,830,632",6.0%,22.6%,15.2%,...,"417,500","311,684","795,129","89,110","1,006,885",231.1,"55,518.93",17,IL,17
KS,"2,911,505","2,913,123","2,853,126","2,853,130",2.0%,2.1%,"2,853,118",6.6%,24.5%,15.4%,...,"77,194","26,127","204,562","21,610","203,401",34.9,"81,758.72",20,KS,20
MS,"2,986,530","2,984,100","2,968,118","2,968,103",0.6%,0.5%,"2,967,297",6.3%,23.9%,15.5%,...,"89,159","74,824","155,094","26,789","198,566",63.2,"46,923.27",28,MS,28


In [9]:
# Creates a collection in the database and inserts documents
db.veilofignorance.insert_many(records)

In [46]:
statesonlydf = dflist.drop(['US'])

#dflist.head()
#dflist.describe()
#dflist['Persons in poverty, percent']
#dflist['Persons in poverty, percent'] = dflist['Persons in poverty, percent'].astype(float)
#dflist['Persons in poverty, percent'] = dflist['Persons in poverty, percent'].str.rstrip('%').astype('float')

#dflist['Persons in poverty, percent'].describe()
statesonlydf['Persons in poverty, percent'] = dflist['Persons in poverty, percent'].str.rstrip('%').astype('float')
statesonlydf.head()
statesonlydf['Persons in poverty, percent'] 

AR    16.4
CA    13.3
IL    12.6
KS    11.9
MS    19.8
OH    14.0
TX    14.7
AL    16.9
IA    10.7
LA    19.7
MN     9.5
MO    13.4
NE    10.8
AZ    14.9
CO    10.3
IN    13.5
MI    14.2
MT    12.5
NY    14.1
OR    13.2
VA    10.6
WY    11.3
NC    14.7
OK    15.8
TN    15.0
WI    11.3
AK    11.1
VT    11.3
ND    10.3
GA    14.9
ME    11.1
RI    11.6
WV    19.1
ID    12.8
SD    13.0
NM    19.7
WA    11.0
PA    12.5
FL    14.0
UT     9.7
KY    17.2
NH     7.7
SC    15.4
NV    13.0
HI     9.5
NJ    10.0
CT     9.6
MD     9.3
MA    10.5
DE    13.6
DC    16.6
Name: Persons in poverty, percent, dtype: float64

In [47]:

#statesonlydf['Persons in poverty, percent'] = statesonlydf['Persons in poverty, percent'].str.rstrip('%').astype('float') / 100.0
statesonlydf['Persons in poverty, percent'].describe()

count    51.000000
mean     13.129412
std       2.912408
min       7.700000
25%      10.900000
50%      13.000000
75%      14.800000
max      19.800000
Name: Persons in poverty, percent, dtype: float64

In [50]:
usonlydf = dflist.loc['US']

In [51]:
usonlydfstatesdflist['Persons in poverty, percent'] = dflist['Persons in poverty, percent'].str.rstrip('%').astype('float')
statesonlydf.head()
statesonlydf['Persons in poverty, percent'] 

Population estimates, July 1, 2018,  (V2018)                                               327,167,434
Population estimates, July 1, 2017,  (V2017)                                               325,719,178
Population estimates base, April 1, 2010,  (V2018)                                         308,758,105
Population estimates base, April 1, 2010,  (V2017)                                         308,758,105
Population, percent change - April 1, 2010 (estimates base) to July 1, 2018,  (V2018)             6.0%
Population, percent change - April 1, 2010 (estimates base) to July 1, 2017,  (V2017)             5.5%
Population, Census, April 1, 2010                                                          308,745,538
Persons under 5 years, percent                                                                    6.1%
Persons under 18 years, percent                                                                  22.6%
Persons 65 years and over, percent                                       

In [69]:
statesonlydf = dflist.drop(['US'])

statesonlydf['Median household income (in 2017 dollars), 2013-2017'] = statesonlydf['Median household income (in 2017 dollars), 2013-2017'].str.replace(",", "").str.replace("$", "").astype("int")

statesonlydf['Median household income (in 2017 dollars), 2013-2017']
statesonlydf['Median household income (in 2017 dollars), 2013-2017'].describe()

count       51.000000
mean     58236.470588
std       9849.810661
min      42009.000000
25%      50934.000000
50%      56570.000000
75%      65391.500000
max      78916.000000
Name: Median household income (in 2017 dollars), 2013-2017, dtype: float64

In [68]:
statesonlydf['Median household income (in 2017 dollars), 2013-2017'] 

AR    43813
CA    67169
IL    61229
KS    55477
MS    42009
OH    52407
TX    57051
AL    46472
IA    56570
LA    46710
MN    65699
MO    51542
NE    56675
AZ    53510
CO    65458
IN    52182
MI    52668
MT    50801
NY    62765
OR    56119
VA    68766
WY    60938
NC    50320
OK    49767
TN    48708
WI    56759
AK    76114
VT    57808
ND    61285
GA    52977
ME    53024
RI    61043
WV    44061
ID    50985
SD    54126
NM    46718
WA    66174
PA    56951
FL    50883
UT    65325
KY    46535
NH    71305
SC    48781
NV    55434
HI    74923
NJ    76475
CT    73781
MD    78916
MA    74167
DE    63036
DC    77649
Name: Median household income (in 2017 dollars), 2013-2017, dtype: int32

In [74]:
statesonlydf['Mean travel time to work (minutes), workers age 16 years+, 2013-2017'].astype(float).describe()

count    51.000000
mean     24.337255
std       3.840050
min      17.000000
25%      21.900000
50%      24.300000
75%      26.850000
max      33.000000
Name: Mean travel time to work (minutes), workers age 16 years+, 2013-2017, dtype: float64